## Scrapy之爬虫启动
　　参考：[Scrapy的抓取流程——CrawlerProcess](https://blog.csdn.net/okm6666/article/details/89160886)<br>
　　　　　[通过核心ＡＰＩ启动单个或多个scrapy爬虫](https://www.jianshu.com/p/add5c59d698a)<br>
　　　　　[Scrapy进阶-命令行的工作原理（以runspider为例）](https://www.jianshu.com/p/8e252b2272d8)<br>
　　　　　[同时运行多个scrapy爬虫的几种方法（自定义scrapy项目命令）](https://www.cnblogs.com/rwxwsblog/p/4578764.html)<br>
　　　　　[scrapy项目下spiders内多个爬虫同时运行](https://blog.csdn.net/beyond_f/article/details/74626451)<br>
　　　　　[python scrapy项目下spiders内多个爬虫同时运行](https://blog.csdn.net/qq_38282706/article/details/80977576)<br>
     
## 一、通过自定义scrapy命令的方式来运行
　　配置说明：https://scrapy-chs.readthedocs.io/zh_CN/1.0/topics/commands.html<br>
1、创建commands目录<br>
>    mkdir commands<br>

　　注意：commands和spiders目录是同级的<br>
2、在commands下面添加一个文件crawlall.py<br>
　　注意：这里主要通过修改scrapy的crawl命令来完成同时执行spider的效果。<br>

In [7]:
from scrapy.commands import ScrapyCommand  
from scrapy.crawler import CrawlerRunner
from scrapy.utils.conf import arglist_to_dict

class Command(ScrapyCommand):
  
    requires_project = True
  
    def syntax(self):  
        return '[options]'  
  
    def short_desc(self):  
        return 'Runs all of the spiders'  

    def add_options(self, parser):
        ScrapyCommand.add_options(self, parser)
        parser.add_option("-a", dest="spargs", action="append", default=[], metavar="NAME=VALUE",
                          help="set spider argument (may be repeated)")
        parser.add_option("-o", "--output", metavar="FILE",
                          help="dump scraped items into FILE (use - for stdout)")
        parser.add_option("-t", "--output-format", metavar="FORMAT",
                          help="format to use for dumping items with -o")

    def process_options(self, args, opts):
        ScrapyCommand.process_options(self, args, opts)
        try:
            opts.spargs = arglist_to_dict(opts.spargs)
        except ValueError:
            raise UsageError("Invalid -a value, use -a NAME=VALUE", print_help=False)

    def run(self, args, opts):
        #settings = get_project_settings()

        spider_loader = self.crawler_process.spider_loader
        for spidername in args or spider_loader.list():
            print("*********cralall spidername************" + spidername)
            self.crawler_process.crawl(spidername, **opts.spargs)

        self.crawler_process.start()

ModuleNotFoundError: No module named 'scrapy'

　　这里主要是用了self.crawler_process.spider_loader.list()方法获取项目下所有的spider，然后利用self.crawler_process.crawl运行spider<br>

　　3、commands命令下添加__init__.py文件<br>

touch __init__.py<br>
　　注意：这一步一定不能省略。我就是因为这个问题折腾了一天。囧。。。就怪自己半路出家的吧。<br>

　　如果省略了会报这样一个异常<br>
>Traceback (most recent call last):<br>
　　File "/usr/local/bin/scrapy", line 9, in \<module\><br>
　　　　load_entry_point('Scrapy==1.0.0rc2', 'console_scripts', 'scrapy')()<br>
　　File "/usr/local/lib/python2.7/site-packages/Scrapy-1.0.0rc2-py2.7.egg/scrapy/cmdline.py", line 122, in execute<br>
　　　　cmds = \_get_commands_dict(settings, inproject)<br>
　　File "/usr/local/lib/python2.7/site-packages/Scrapy-1.0.0rc2-py2.7.egg/scrapy/cmdline.py", line 50, in \_get_commands_dict<br>
　　　　cmds.update(\_get_commands_from_module(cmds_module, inproject))<br>
　　File "/usr/local/lib/python2.7/site-packages/Scrapy-1.0.0rc2-py2.7.egg/scrapy/cmdline.py", line 29, in \_get_commands_from_module<br>
　　　　for cmd in \_iter_command_classes(module):<br>
　　File "/usr/local/lib/python2.7/site-packages/Scrapy-1.0.0rc2-py2.7.egg/scrapy/cmdline.py", line 20, in \_iter_command_classes<br>
　　　　for module in walk_modules(module_name):<br>
　　File "/usr/local/lib/python2.7/site-packages/Scrapy-1.0.0rc2-py2.7.egg/scrapy/utils/misc.py", line 63, in walk_modules<br>
　　　　mod = import_module(path)<br>
　　File "/usr/local/lib/python2.7/importlib/\_\_init__.py", line 37, in import_module<br>
　　　　\_\_import__(name)<br>
ImportError: No module named commands<br>

　　一开始怎么找都找不到原因在哪。耗了我一整天，后来到http://stackoverflow.com/上得到了网友的帮助。再次感谢万能的互联网，要是没有那道墙该是多么的美好呀！扯远了，继续回来<br>

　　4、settings.py目录下创建setup.py（这一步去掉也没影响，不知道官网帮助文档这么写有什么具体的意义。<br>

>from setuptools import setup, find_packages<br>
<br>
setup(name='scrapy-mymodule',<br>
　　entry_points={<br>
　　　　'scrapy.commands': [<br>
　　　　　　'crawlall=cnblogs.commands:crawlall',<br>
　　　　],<br>
　　},<br>
)<br>

　　这个文件的含义是定义了一个crawlall命令，cnblogs.commands为命令文件目录，crawlall为命令名。<br>

　　5. 在settings.py中添加配置：<br>
>COMMANDS_MODULE = 'cnblogs.commands'<br>

　　6. 运行命令scrapy crawlall<br>